In [ ]:
from dataclasses import fields
from os import path
from typing import Union, List, Dict

import numpy as np
import yaml

from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.ConversionAnalyzer.ErrorAnalysisResult import ErrorAnalysisResult
from OpenSCENARIO2CR.ConversionAnalyzer.STLMonitor import STLMonitorResult
from OpenSCENARIO2CR.Osc2CrConverter import EFailureReason
from OpenSCENARIO2CR.util.ConversionStatistics import ConversionStatistics

In [ ]:
stats_dir = "results/"

with open(path.join(stats_dir, "statistics.yml"), "r") as stats_file:
    imported_statistics = yaml.safe_load(stats_file)

In [ ]:
def analyze_statistics(statistics: Dict):
    counts = {}

    def count(name: str, amount: int = 1):
        if name in counts:
            counts[name] += amount
        else:
            counts[name] = amount

    def perc(
            description: str,
            part_str: Union[str, List[str]],
            total_str: Union[str, List[str]],
            invert: bool = False
    ):
        if isinstance(part_str, str):
            part = counts.get(part_str, 0)
        else:
            part = sum([counts.get(single_part, 0) for single_part in part_str])
        if isinstance(total_str, str):
            total = counts.get(total_str, 0)
        else:
            total = sum([counts.get(single_total, 0) for single_total in total_str])
        if invert:
            part = total - part
        if total != 0:
            res = f"{100 * part / total:5.1f} %"
        else:
            res = "  NaN  "
        print(f"{description:<50s} {res} ({part}/{total})")

    times = []
    rules = [rule.name for rule in fields(STLMonitorResult)]

    for scenario_path, result in statistics.items():
        count("total")
        if result[0] == "success":
            count("success")
            stats = ConversionStatistics.from_dict(result[1])
            times.append(stats.sim_time)
            count("obstacle_total", stats.num_obstacle_conversions)
            count("obstacle_failed", len(stats.failed_obstacle_conversions))
            if EAnalyzer.STL_MONITOR in stats.analysis:
                count("compliance_total_scenario")
                scenario_run = True
                scenario_compliant = True
                rules_run = {rule: True for rule in rules}
                rules_compliant = {rule: True for rule in rules}
                for vehicle_name, all_monitor_result in stats.analysis[EAnalyzer.STL_MONITOR].items():
                    count("compliance_total_vehicle")
                    vehicle_run = True
                    vehicle_compliant = True
                    for rule in rules:
                        count(f"compliance_total_vehicle_" + rule.upper())
                        if isinstance(all_monitor_result, ErrorAnalysisResult) or getattr(all_monitor_result,
                                                                                          rule) is None:
                            scenario_run = False
                            scenario_compliant = False
                            vehicle_run = False
                            vehicle_compliant = False
                            rules_run[rule] = False
                            rules_compliant[rule] = False
                            continue
                        count("compliance_run_vehicle_" + rule.upper())
                        if all([value >= 0.0 for value in getattr(all_monitor_result, rule)]):
                            count("compliance_compliant_vehicle_" + rule.upper())
                        else:
                            scenario_compliant = False
                            vehicle_compliant = False
                            rules_compliant[rule] = False
                    if vehicle_run:
                        count("compliance_run_vehicle")
                    if vehicle_compliant:
                        count("compliance_compliant_vehicle")
                if scenario_run:
                    count("compliance_run_scenario")
                if scenario_compliant:
                    count("compliance_compliant_scenario")

                for rule in rules:
                    count("compliance_total_scenario_" + rule.upper())
                    if rules_run[rule]:
                        count("compliance_run_scenario_" + rule.upper())
                    if rules_compliant[rule]:
                        count("compliance_compliant_scenario_" + rule.upper())


        elif result[0] == "failure":
            count("failed")
            count(f"failed_{result[1]}")
        elif result[0] == "error":
            count("error")

    print(f"{'Total num scenarios':<50s} {counts['total']:5d}")
    print(f"{'Average time':<50s} {np.mean(times):}")
    print("-" * 58)
    perc("Conversion success rate", "success", "total")
    perc("Conversion failure rate", "failed", "total")
    for reason in EFailureReason:
        perc(f" | {reason.name}", f"failed_{reason.name}", "failed")
    perc("Conversion error rate", "error", "total")
    print("-" * 58)
    perc("Obstacle conversion rate", "obstacle_failed", "obstacle_total", invert=True)
    print("-" * 58)
    print("CommonRoad STL Monitor Results, granularity: scenario")
    perc("run rate", "compliance_run_scenario", "compliance_total_scenario")
    perc("compliant rate of total", "compliance_compliant_scenario", "compliance_total_scenario")
    perc("compliant rate of run", "compliance_compliant_scenario", "compliance_run_scenario")
    for rule in rules:
        perc(f"{rule.upper()} run rate", f"compliance_run_scenario_{rule.upper()}",
             f"compliance_total_scenario_{rule.upper()}")
        perc(f"{rule.upper()} compliant rate of total", f"compliance_compliant_scenario_{rule.upper()}",
             f"compliance_total_scenario_{rule.upper()}")
        perc(f"{rule.upper()} compliant rate of run", f"compliance_compliant_scenario_{rule.upper()}",
             f"compliance_run_scenario_{rule.upper()}")
    print("-" * 58)
    print("CommonRoad STL Monitor Results, granularity: vehicle")
    perc("run rate", "compliance_run_vehicle", "compliance_total_vehicle")
    perc("compliant rate of total", "compliance_compliant_vehicle", "compliance_total_vehicle")
    perc("compliant rate of run", "compliance_compliant_vehicle", "compliance_run_vehicle")
    for rule in rules:
        perc(f"{rule.upper()} run rate", f"compliance_run_vehicle_{rule.upper()}",
             f"compliance_total_vehicle_{rule.upper()}")
        perc(f"{rule.upper()} compliant rate of total", f"compliance_compliant_vehicle_{rule.upper()}",
             f"compliance_total_vehicle_{rule.upper()}")
        perc(f"{rule.upper()} compliant rate of run", f"compliance_compliant_vehicle_{rule.upper()}",
             f"compliance_run_vehicle_{rule.upper()}")


def print_exception_tracebacks(statistics: Dict, compressed=True):
    errors: Dict[ErrorAnalysisResult, int] = {}
    for scenario_path, result in statistics.items():
        if result[0] == "error":
            error = ErrorAnalysisResult.from_dict(result[1])
            if not compressed:
                print(f"{scenario_path}\n{error.traceback_text}")
                print("\n" * 3)
            elif error in errors:
                errors[error] += 1
            else:
                errors[error] = 1

    for error, count in errors.items():
        print(f"{count}\n{error.traceback_text}")
        print("\n" * 3)


def print_exception_tracebacks_for_analyzer(statistics: Dict, analyzer: EAnalyzer, compressed=True):
    errors: Dict[ErrorAnalysisResult, int] = {}
    for scenario_path, result in statistics.items():
        if scenario_path == "results/2022-09-21_08:32:45/ALKS_Scenario_4.6_2_LateralDetectionRange_TEMPLATE":
            print("A")
        if result[0] == "success":
            stats = ConversionStatistics.from_dict(result[1])
            if analyzer in stats.analysis:
                for vehicle_name, analyzer_result in stats.analysis[analyzer].items():
                    if isinstance(analyzer_result, ErrorAnalysisResult):
                        if not compressed:
                            print(f"{stats.source_file}\n{analyzer_result.traceback_text}")
                        elif analyzer_result in errors:
                            errors[analyzer_result] += 1
                        else:
                            errors[analyzer_result] = 1

    for error, count in errors.items():
        print(f"{count}\n{error.traceback_text}")
        print("\n" * 3)


In [ ]:
analyze_statistics(imported_statistics)

In [ ]:
print_exception_tracebacks(imported_statistics, compressed=True)

In [ ]:
print_exception_tracebacks_for_analyzer(imported_statistics, EAnalyzer.STL_MONITOR, compressed=True)